In [49]:
import numpy as np

In [71]:
class LinearRegression:


    def __init__(self, lr = 0.01, epochs = 1000, tol = 1e-12, batch_size = 100_000, seed = np.random.seed(42)) -> None:
        
        if lr <= 0:
            raise ValueError("learning rate must be positive")
        
        if epochs <= 0:
            raise ValueError("epoch must be positive")
        
        self.lr = lr
        self.tol = tol
        self.epochs = epochs
        self.loss = np.zeros(shape = (self.epochs, ))
        self.seed = seed
        self.batch_size = batch_size
    
    def batch_generator(self, X, y):

        num_samples, _ = X.shape
        indices = np.arange(num_samples)
        np.random.shuffle(indices)

        for start in np.arange(0, num_samples, self.batch_size):

            end = min(start + self.batch_size, num_samples)
            yield X[start : end], y[start : end]
        

    def fit(self, X, y) -> None:

        self.m, self.n = X.shape
        self.x = X
        self.y = y

        if self.y.ndim == 1:

            self.weights = np.random.uniform(-0.5, 0.5, size = (self.n, ))
            self.bias = np.random.uniform(-0.5, 0.5)
        
        else:

            self.weights = np.random.uniform(-0.5, 0.5, size = (self.n, 1))
            self.bias = np.random.uniform(-0.5, 0.5)
        
        
        for epoch in range(self.epochs):

            total_loss = 0
            

            for x_batch, y_batch in self.batch_generator(self.x, self.y):


                pred = self.predict(x_batch)
                batch_loss = np.mean(np.square(y_batch - pred))
                total_loss += batch_loss

                self.weights -= self.lr * (-2 / self.batch_size) * np.dot(x_batch.T, y_batch - pred)
                self.bias -= self.lr * (-2 / self.batch_size) * np.sum(y_batch - pred)

            self.loss[epoch] = total_loss / (self.m / self.batch_size)

            if epoch % 10 == 0:
                print(f"epoch {epoch}/{self.epochs} -- loss {self.loss[epoch]:.2f}")

            if epoch >= 1 and np.abs(self.loss[epoch] - self.loss[epoch - 1]) < self.tol:

                break
    

    def predict(self, X):

        return np.dot(X, self.weights) + self.bias